# 第7章: データベース

In [1]:
%system curl -O http://www.cl.ecei.tohoku.ac.jp/nlp100/data/artist.json.gz

['  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current',
 '                                 Dload  Upload   Total   Spent    Left  Speed',
 '',
 '  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0',
 ' 18 46.9M   18 8690k    0     0  9529k      0  0:00:05 --:--:--  0:00:05 9528k',
 ' 41 46.9M   41 19.6M    0     0  10.2M      0  0:00:04  0:00:01  0:00:03 10.2M',
 ' 65 46.9M   65 30.8M    0     0  10.6M      0  0:00:04  0:00:02  0:00:02 10.6M',
 ' 89 46.9M   89 42.0M    0     0  10.7M      0  0:00:04  0:00:03  0:00:01 10.7M',
 '100 46.9M  100 46.9M    0     0  10.8M      0  0:00:04  0:00:04 --:--:-- 10.8M']

## 60. KVSの構築

In [9]:
import gzip
import json
import unicodedata
import redis

db = redis.Redis()

with gzip.open('artist.json.gz') as fd:
    for line in fd:
        data_json = json.loads(line)
        key = unicodedata.normalize('NFKC', data_json['name']) + '\t' + str(data_json['id'])
        value = data_json.get('area', '')
        db.set(key, value)

## 61. KVSの検索

In [13]:
artist = input('> ')
for key in db.keys(artist + '\t*'):
    print(key.decode('utf8'), db.get(key).decode('utf8'))

> Oasis
Oasis	286198 United States
Oasis	377879 United Kingdom
Oasis	20660 United Kingdom


## 62. KVS内の反復処理

In [14]:
count = 0
for key in db.keys('*'):
    if db.get(key) == b'Japan':
        count += 1
print(count)

22893


## 63. オブジェクトを値に格納したKVS

In [19]:
db.flushdb()

with gzip.open('artist.json.gz') as fd:
    for line in fd:
        data_json = json.loads(line)
        key = unicodedata.normalize('NFKC', data_json['name']) + '\t' + str(data_json['id'])
        for tag in data_json.get('tags', []):
            db.hset(key, tag['value'], tag['count'])

In [20]:
artist = input('> ')
for key in db.keys(artist + '\t*'):
    print(key.decode('utf8'), db.hgetall(key))

> Oasis
Oasis	377879 {b'morning glory': b'1', b'oasis': b'1'}
Oasis	20660 {b'rock': b'1', b'britpop': b'3', b'british': b'4', b'uk': b'1', b'britannique': b'1', b'rock and indie': b'1', b'england': b'1', b'manchester': b'1'}


## MongoDBの構築

In [1]:
import rethinkdb as r

r.connect().repl()
r.db("test").table_create("artists").run()

ReqlOpFailedError: Table `test.artists` already exists in:
r.db('test').table_create('artists')
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [2]:
import gzip
import json
import unicodedata

artists = []
with gzip.open('artist.json.gz') as fd:
    for (i, line) in enumerate(fd):
        data_json = json.loads(line)
        key = unicodedata.normalize('NFKC', data_json['name']) + '\t' + str(data_json['id'])
        artists.append({'name': key,
                       'area': data_json.get('area', ''),
                       'aliases.name': [tag['name'] for tag in data_json.get('aliases', [])],
                       'tags.value': [tag['value'] for tag in data_json.get('tags', [])],
                       'rating.value': data_json.get('rating', {}).get('value', 0)})
        if i % 100000 == 0:
            r.table("artists").insert(artists).run()
            artists = []
r.table("artists").insert(artists).run()

{'deleted': 0,
 'errors': 0,
 'generated_keys': ['d33ba6eb-8e62-4d3e-b939-d37232563004',
  '25be4870-b74b-48b5-b078-c10385717b48',
  '40b08443-f2d2-4109-bb42-dbcde0c65e07',
  'a0415956-90bd-4532-8510-cb2a5c5a9869',
  '437c27b2-8598-400c-a0b9-ab28a03733f7',
  '42131a98-7b06-42e1-b978-28d7e81a4227',
  '950ab889-903b-4d07-8ab7-a616f347c347',
  '0e587fa6-e28a-44eb-afa1-81835dd0be9c',
  '154e8c95-be8e-4800-8c6a-c0261320ee20',
  'f2172093-f3eb-4085-a9ab-0b7925cb3eab',
  '28451ba4-f368-4c79-a385-e58cd781f146',
  '439c0e0c-a5af-428f-b98d-998be12c8dfe',
  '16806058-7fa7-4503-b609-19e02888fee6',
  'df0a7a02-d429-4ae8-90f0-d08757259ae3',
  '06c55ba1-2537-4995-97c2-62f7e288b34f',
  'd6f58621-33c2-4ba0-894d-1a978e59a942',
  '07ffaff6-d2c6-4644-9a85-06a7899df115',
  'c06a1c3f-4658-4149-997b-8391d3a47711',
  '61817a51-8339-41a2-a0f2-e36b9b7e5638',
  '69f82d7e-6e2c-44a4-bd16-7816b4906a23',
  'a1f3a30d-9aa3-4060-8029-592ca86d9880',
  'b3b05f32-fe57-4d05-920a-d79d5985870e',
  '3df44694-a2c2-47a7-b2c0-21

## 65. MongoDBの検索

In [3]:
artist = input('> ')
cursor = r.table("artists").filter(r.row['name'].match('^' + artist + '\t*')).run()
for document in cursor:
    print(document)

> Queen
{'aliases.name': ['Queen & Paul Rodgers', 'Queen and Paul Rodgers', 'Queen + PR'], 'area': '', 'id': '21d998b9-8dc1-46ef-977b-01549992f97a', 'name': 'Queen + Paul Rodgers\t286666', 'rating.value': 0, 'tags.value': []}
{'aliases.name': [], 'area': 'United States', 'id': '1952613c-4b17-485b-b6a2-ce56e1b06065', 'name': 'Queen Kapiʻolani\t984046', 'rating.value': 0, 'tags.value': []}
{'aliases.name': [], 'area': '', 'id': '1861f542-6707-486f-acbf-1ef2bf70d34c', 'name': 'Queen Pen\t100317', 'rating.value': 0, 'tags.value': []}
{'aliases.name': [], 'area': '', 'id': '14d7848a-b832-4a26-91bb-cba2d51ca04a', 'name': 'Queen of Hearts\t136639', 'rating.value': 0, 'tags.value': []}
{'aliases.name': [], 'area': '', 'id': '03ae96ac-e3c9-461b-8194-c5c5658c5706', 'name': 'Queen of Japan\t78402', 'rating.value': 0, 'tags.value': []}
{'aliases.name': [], 'area': '', 'id': '00df4eee-ae1a-4fbf-94d7-03622b5e2e15', 'name': 'Queen Mab\t945757', 'rating.value': 0, 'tags.value': []}
{'aliases.name': []

## 66. 検索件数の取得

In [4]:
cursor = r.table("artists").filter(r.row["area"] == 'Japan').run()
print(sum([1 for document in cursor]))

22821


## 67. 複数のドキュメントの取得

In [5]:
artist = 'スマップ'
cursor = r.table("artists").run()

for document in cursor:
    if artist in document['aliases.name']:
        print(document)

{'aliases.name': ['スマップ'], 'area': 'Japan', 'id': '735aa84f-a23d-4417-ab61-00a8616378f9', 'name': 'SMAP\t265728', 'rating.value': 0, 'tags.value': ['likedis auto']}


## 68. ソート

In [6]:
cursor = r.table("artists").filter(lambda artist: artist['rating.value'] > 0).run()

for doc in sorted([ doc for doc in cursor if 'dance' in doc['tags.value']],
                  key=lambda x: x['rating.value'], reverse=True)[:10]:
    print(doc)

{'aliases.name': [], 'area': 'Sweden', 'id': '03174065-1dc8-4ff6-9da1-01fc68718b4c', 'name': 'Army of Lovers\t1865', 'rating.value': 100, 'tags.value': ['dance', 'pop', 'eurodance']}
{'aliases.name': [], 'area': 'United States', 'id': '073a7c6b-5082-44f6-a8fb-7690670bf83c', 'name': 'The Broadcasters\t928847', 'rating.value': 100, 'tags.value': ['album', 'cd', 'dance', 'dance music', 'deep', 'electro', 'electronica', 'ep', 'fun', 'funk', 'guardian', 'hit', 'intelligent', 'interesting', 'live', 'lyrics', 'music', 'pop', 'pop music', 'smart', 'song', 'songs', 'symmetry in the making', 'techno', 'the broadcast', 'the broadcasters', 'the imposter', 'the imposter and his electric orchestra', 'top hits', "you think they're yours"]}
{'aliases.name': [], 'area': '', 'id': '06b70f6a-3e81-4261-a1bd-dfa9660ccb0a', 'name': 'Destinee & Paris\t729051', 'rating.value': 100, 'tags.value': ['dance', 'clique girlz', 'rock', 'pop']}
{'aliases.name': ['David Ghetta', 'Davie Guetta', 'David Guette'], 'area'

In [ ]:
from flask import Flask, Markup, request
app = Flask(__name__)

HTML = '''<html>
    <head>
        <title>言語処理100本ノック2015 問題69</title>
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
    </head>
    <body>
        <form method="POST" action="/">
            名前、別名：<input type="text" name="name" size="20"/><br />
            タグ：<input type="text" name="tag" size="20"/><br />
            <input type="submit" value="検索"/>
        </form>
        %s
    </body>
</html>'''

def sort(cursor):
    return [doc for doc in sorted([ doc for doc in cursor],
            key=lambda x: x['rating.value'], reverse=True)][:10]

def search(artist, tag):
    contents = ''
    if artist:
        cursor = r.table("artists").filter(r.row['name'].match('^' + artist + '\t*')).run()
        for doc in sort(cursor):
            contents += str(doc) + '<br>\n'
    if tag:
        cursor = r.table("artists").run()
        cursor = [doc for doc in cursor if tag in doc['tags.value']]
        for doc in sort(cursor):
            contents += str(doc) + '<br>\n'
    return contents

@app.route("/", methods=['GET', 'POST'])
def contents():
    message = ''
    contents = ''
    artist = request.form.get('name')
    tag = request.form.get('tag')
    if artist or tag:
        contents = search(artist, tag)
    return Markup(HTML % (contents))

if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/May/2017 15:54:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2017 15:54:40] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2017 15:54:50] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2017 15:55:10] "GET / HTTP/1.1" 200 -
